<a href="https://colab.research.google.com/github/SediKOR/Ybigta_assignment/blob/main/RNN_%ED%95%84%EC%88%98_23%EA%B8%B0_%EC%9D%B4%EC%84%B8%EC%A7%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Recurrent Neural Networks - 필수 과제**

**LSTM**을 구현해봅시다!
<br><br><br>
**필요 사전 지식**:

- <u>PyTorch</u> (선택 과제 1)

<br>

**추가 사전 지식**: (알면 좋으나 몰라도 괜찮음)

- <u>Tokenization</u>, <u>Word Embedding</u> (선택 과제 2)

<br><br><br><br><br>

In [1]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00


In [2]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader

from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset

from tqdm import tqdm

<br><br>

[Hugging Face](https://huggingface.co)에서 [Rotten Tomatoes dataset](https://huggingface.co/datasets/rotten_tomatoes)과 [pretrained BERT](https://huggingface.co/bert-base-uncased)의 tokenizer를 가져오겠습니다.

또 학습 부담을 줄이기 위해 pretrained BERT에 내장된 word embedding layer의 weight도 가져옵시다.

In [3]:
# https://huggingface.co/datasets/rotten_tomatoes
dataset = load_dataset("rotten_tomatoes")

# https://huggingface.co/bert-base-uncased
pretrained_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
pretrained_embeddings = AutoModel.from_pretrained("bert-base-uncased").embeddings.word_embeddings

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

<br><br>

기본 BERT는 token을 768차원 벡터로 embedding합니다. 우리의 작은 dataset과 작은 모델에게 768차원은 부담스러우니 PCA를 사용해 64차원으로 줄여줍시다.

In [4]:
nano_embed = torch.pca_lowrank(pretrained_embeddings.weight.detach(), q=64)[0]

<br><br>

그런데 무작정 64차원으로 줄여도 되는 걸까요? BERT의 d_model이 괜히 768도 아닐 테고, 정보의 손실이 아주 클 것 같은데 말입니다.

궁금하니 코사인 유사도로 축소된 embedding layer에 token들의 정보가 그럭저럭 잘 남아있는지 확인해봅시다.

In [5]:
cos = (nano_embed @ nano_embed.T) / (nano_embed.abs() @ nano_embed.abs().T)

In [6]:
# word에 다양한 값을 넣어보세요! tokenizer의 vocab에 없는 token에 대해서는 빈 list가 뜹니다.
word = "jackson"

([*map(pretrained_tokenizer.decode, cos[pretrained_tokenizer.vocab[word]].argsort(descending=True)[1:21])] if word in pretrained_tokenizer.vocab else [])

['answered',
 'went',
 'then',
 'tilted',
 'motivation',
 'going',
 'really',
 'go',
 'collins',
 'answers',
 'down',
 'mason',
 '##bs',
 'him',
 '##ash',
 'prescott',
 'elias',
 'though',
 'enough',
 'me']

꽤 잘 남아있는 것 같습니다.

(TMI: 조금 더 욕심을 부려 한번 32차원으로 줄여보면 무시하기 어려운 정보의 손실을 체감할 수 있습니다.)

<br><br>

이제 LSTM을 구현합시다! 사실 원래 BiLSTM으로 하려고 했는데 underfitting이 심해서 그냥 plain LSTM으로 준비했습니다.

<br><br><br><br>
#### <span style="color:red">**<u>Q1.</u>**</span>

`class LSTMCell`의 빈칸을 채우세요.

In [7]:
import torch
import torch.nn as nn

class LSTMCell(nn.Module):
    def __init__(self, d_x, d_h):
        super().__init__()
        d_stack = d_x + d_h
        ######################### YOUR CODE START #########################

        self.W_f = nn.Linear(d_stack, d_h)
        self.b_f = nn.Parameter(torch.zeros(d_h))

        self.W_i = nn.Linear(d_stack, d_h)
        self.b_i = nn.Parameter(torch.zeros(d_h))

        self.W_C = nn.Linear(d_stack, d_h)
        self.b_C = nn.Parameter(torch.zeros(d_h))

        self.W_o = nn.Linear(d_stack, d_h)
        self.b_o = nn.Parameter(torch.zeros(d_h))

        ########################## YOUR CODE END ##########################
        self.W_f = nn.Linear(d_stack, d_h)
        self.W_i = nn.Linear(dim1, dim2)
        self.W_C = nn.Linear(dim3, dim4)
        self.W_o = nn.Linear(dim5, dim6)


    # forward는 t-1의 h_{t-1}, C_{t-1}과 t의 x_t를 입력으로 받아 계산합니다.

    def forward(self, x, h, C):
        stack = torch.cat([x, h])
        ######################### YOUR CODE START #########################

        f = torch.sigmoid(self.W_f(stack) + self.b_f)
        i = torch.sigmoid(self.W_i(stack) + self.b_i)
        C_ = torch.tanh(self.W_C(stack) + self.b_C)

        C_t = f * C + i * C_

        o = torch.sigmoid(self.W_o(stack) + self.b_o)
        h_t = o * torch.tanh(C_t)

        ########################## YOUR CODE END ##########################
        return h_t, C_t


In [8]:
import torch
import torch.nn as nn

class LSTMCell(nn.Module):
    def __init__(self, d_x, d_h):
        super().__init__()
        d_stack = d_x + d_h
        ######################### YOUR CODE START #########################

        self.W_f = nn.Linear(d_stack, d_h)
        self.W_i = nn.Linear(d_stack, d_h)
        self.W_C = nn.Linear(d_stack, d_h)
        self.W_o = nn.Linear(d_stack, d_h)

        self.b_f = nn.Parameter(torch.zeros(d_h))
        self.b_i = nn.Parameter(torch.zeros(d_h))
        self.b_C = nn.Parameter(torch.zeros(d_h))
        self.b_o = nn.Parameter(torch.zeros(d_h))

        dim1 = d_stack
        dim2 = d_h
        dim3 = d_stack
        dim4 = d_h
        dim5 = d_stack
        dim6 = d_h


        ########################## YOUR CODE END ##########################
        self.W_f = nn.Linear(d_stack, d_h)
        self.W_i = nn.Linear(dim1, dim2)
        self.W_C = nn.Linear(dim3, dim4)
        self.W_o = nn.Linear(dim5, dim6)


    # forward는 t-1의 h_{t-1}, C_{t-1}과 t의 x_t를 입력으로 받아 계산합니다.

    def forward(self, x, h, C):
        stack = torch.cat([x, h])
        ######################### YOUR CODE START #########################

        f = torch.sigmoid(self.W_f(stack) + self.b_f)
        i = torch.sigmoid(self.W_i(stack) + self.b_i)
        C_ = torch.tanh(self.W_C(stack) + self.b_C)

        C_t = f * C + i * C_

        o = torch.sigmoid(self.W_o(stack) + self.b_o)
        h_t = o * torch.tanh(C_t)

        ########################## YOUR CODE END ##########################
        return h_t, C_t


In [9]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, d_out, pretrained_embeddings):
        super().__init__()
        vocab_size = pretrained_embeddings.shape[0]
        d_h = d_model = pretrained_embeddings.shape[1]

        self.embed = nn.Embedding(vocab_size, d_model, _weight=pretrained_embeddings.clone()) # word embedding layer
        self.cell = LSTMCell(d_x=d_model, d_h=d_h) # LSTM cell
        self.out = nn.Linear(d_h, d_out, bias=True) # output layer

        self.h_init = nn.Parameter(torch.zeros(d_h), requires_grad=False) # initial h
        self.C_init = nn.Parameter(torch.zeros(d_h), requires_grad=False) # initial C

    def forward(self, input_ids):
        embedded = self.embed(input_ids).squeeze()

        h = self.h_init.clone() # h 초기화
        C = self.C_init.clone() # C 초기화
        for x in embedded:
            h, C = self.cell(x, h, C) # iterate over embedded sequence

        return self.out(h).squeeze() # last hidden state를 output layer에 통과시킨 값을 반환

<br><br><br><br>
#### <span style="color:red">**<u>Q2.</u>**</span>

Test accuracy가 0.7 이상이 되도록 모델을 훈련시키세요.

In [10]:
######################### START OF YOUR CODE #########################

# 필요에 따라 바꿔도 됩니다.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

########################## END OF YOUR CODE ##########################

model = LSTM(vocab_size=pretrained_tokenizer.vocab_size, d_out=1, pretrained_embeddings=nano_embed).to(device)

In [11]:
######################### START OF YOUR CODE #########################

# learning rate을 적절히 수정해보세요.
lr = 1e-4

########################## END OF YOUR CODE ##########################

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [12]:
train_loader = DataLoader(dataset["train"], shuffle=True)

In [16]:
######################### START OF YOUR CODE #########################

# 필요에 따라 바꿔도 됩니다.
num_print = 500
num_batch = 10

########################## END OF YOUR CODE ##########################


# train

save_l = 0
optimizer.zero_grad()
for i, data in enumerate(tqdm(train_loader)):
    text, label = data["text"][0], data["label"][0]
    input_ids = pretrained_tokenizer.encode(text, return_tensors="pt").to(device)
    y_pred = model(input_ids)

    label = label.to(device) * 1.
    loss = criterion(y_pred.sigmoid(), label)
    loss.backward()

    if not (i+1)%num_batch:
        optimizer.step()
        optimizer.zero_grad()

    save_l += loss.item()
    if not (i+1)%num_print:
        print(f"{i+1:>5} iter: {save_l/num_print}")
        save_l = 0

  6%|▌         | 504/8530 [00:14<03:46, 35.39it/s]

  500 iter: 0.6921236503124237


 12%|█▏        | 1004/8530 [00:28<03:17, 38.20it/s]

 1000 iter: 0.6923209648132325


 18%|█▊        | 1504/8530 [00:43<03:18, 35.47it/s]

...

 94%|█████████▍| 8006/8530 [03:49<00:12, 41.22it/s]

 8000 iter: 0.5799672658741474


100%|█████████▉| 8503/8530 [04:04<00:00, 32.42it/s]

 8500 iter: 0.5945853234678506


100%|██████████| 8530/8530 [04:05<00:00, 34.75it/s]


In [17]:
test_loader = DataLoader(dataset["test"], shuffle=True)


# test

res = torch.tensor(0)
with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        text, label = data["text"][0], data["label"][0]
        input_ids = pretrained_tokenizer.encode(text, return_tensors="pt").to(device)
        y_pred = model(input_ids)
        res += ((1 if y_pred > 0 else 0) == label)

print("Test accuracy:", res.item() / dataset["test"].num_rows)

100%|██████████| 1066/1066 [00:10<00:00, 100.60it/s]

Test accuracy: 0.7457786116322702


In [18]:
# 관찰용
# n 값을 바꿔가며 훈련시킨 모델의 예측값을 구경해보세요
n = 300

print(dataset["test"][n])
with torch.no_grad():
    print(model(pretrained_tokenizer.encode(dataset["test"][n]["text"], return_tensors="pt").to(device)).sigmoid().item())

{'text': "that death is merely a transition is a common tenet in the world's religions . this deeply spiritual film taps into the meaning and consolation in afterlife communications .", 'label': 1}
0.5306731462478638
